<a href="https://colab.research.google.com/github/vidyabandgar97/A2A/blob/main/detectron2_500_03_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/detectron2_500/

/content/drive/MyDrive/detectron2_500


In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 23 kB/s 
     |████████████████████████████████| 6.6 MB 31.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.5.0+cu101 which is incompatible.
     |████████████████████████████████| 274 kB 5.2 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=e5fcf6842f471b4b059e9b28ccbe42

In [4]:
!pip install torch==1.9.0 torchvision==0.4.1

     |████████████████████████████████| 831.4 MB 2.8 kB/s 
     |████████████████████████████████| 10.1 MB 13.1 MB/s 
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==1.9.0 and torchvision==0.4.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==1.9.0
    torchvision 0.4.1 depends on torch==1.3.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


In [5]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2 MB 773 kB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210825-py3-none-any.whl size=60661 sha256=9d92eec834045b764cf7d1ca74fb993f0db368b32a0bae36d2d935eb9b1de1d6
  Stored in directory: /root/.cache/pip/wheels/53/c4/f8/c4cb07f135845218b019b4a55d8a0470a0f21ee13f8dcd16be
Successfully built fvcore


In [6]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer

In [7]:
#if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train2", {}, "/content/drive/MyDrive/detectron2_500/train.json", "/content/drive/MyDrive/detectron2_500/Dataset")
register_coco_instances("my_dataset_val2", {}, "/content/drive/MyDrive/detectron2_500/test.json", "/content/drive/MyDrive/detectron2_500/test_data")
register_coco_instances("my_dataset_test2", {}, "/content/drive/MyDrive/detectron2_500/test.json", "/content/drive/MyDrive/detectron2_500/test_data")

In [8]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import transforms as T

from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)

from detectron2.data import DatasetMapper   # the default mapper

In [9]:
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_train_loader(cls, cfg):
    return build_detection_train_loader(
        cfg, 
        mapper=DatasetMapper(
            cfg, 
            is_train=True, 
            augmentations=[
                           T.Resize((1024, 1024)), 
                           T.HFlipTransform(1024)
                           ]
        )
    )

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [10]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
import os

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/detectron2_500/output_detect"
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.merge_from_file(model_zoo.get_config_file("Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train2",)
cfg.DATASETS.TEST = ("my_dataset_val2",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/mask_rcnn_R_50_FPN_3x_dconv_c3-c5.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 66000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()
#trainer.test()


[09/03 08:08:39 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [11]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[09/03 10:51:35 d2.data.datasets.coco]: Loaded 190 images in COCO format from /content/drive/MyDrive/detectron2_500/test.json
[09/03 10:51:35 d2.data.common]: Serializing 190 elements to byte tensors and concatenating them all ...
[09/03 10:51:35 d2.data.common]: Serialized dataset takes 0.34 MiB
[09/03 10:51:35 d2.evaluation.evaluator]: Start inference on 190 images
[09/03 10:51:39 d2.evaluation.evaluator]: Inference done 11/190. 0.3335 s / img. ETA=0:01:01
[09/03 10:51:44 d2.evaluation.evaluator]: Inference done 25/190. 0.3456 s / img. ETA=0:00:58
[09/03 10:51:49 d2.evaluation.evaluator]: Inference done 39/190. 0.3522 s / img. ETA=0:00:54
[09/03 10:51:54 d2.evaluation.evaluator]: Inference done 53/190. 0.3525 s / img. ETA=0:00:49
[09/03 10:51:59 d2.evaluation.evaluator]: Inference done 67/190. 0.3527 s / img. ETA=0:00:44
[09/03 10:52:05 d2.evaluation.evaluator]: Inference done 81/190. 0.3548 s / img. ETA=0:00:39
[09/03 10:52:10 d2.evaluation.evaluator]: Inference done 95/190. 0.3565 

OrderedDict([('bbox',
              {'AP': 30.028954867070958,
               'AP-aeroplane': 54.492201871100164,
               'AP-car': 16.518143172778075,
               'AP-chair': 21.81938670292362,
               'AP-cow': 50.554033712020306,
               'AP-person': 11.101298937862536,
               'AP-traffic_light': 25.68866480574103,
               'AP50': 45.9491811115916,
               'AP75': 31.514182270883616,
               'APl': 45.27411067071747,
               'APm': 41.31765659109794,
               'APs': 15.323416365796074}),
             ('segm',
              {'AP': 24.213020765128487,
               'AP-aeroplane': 39.783182438186024,
               'AP-car': 14.815570284965865,
               'AP-chair': 16.688287929509446,
               'AP-cow': 41.185953406719165,
               'AP-person': 9.33184244640012,
               'AP-traffic_light': 23.4732880849903,
               'AP50': 42.76290697621386,
               'AP75': 24.71819873424319,
    

In [14]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_0065999.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[09/03 11:02:48 d2.data.datasets.coco]: Loaded 190 images in COCO format from /content/drive/MyDrive/detectron2_500/test.json
[09/03 11:02:48 d2.data.common]: Serializing 190 elements to byte tensors and concatenating them all ...
[09/03 11:02:48 d2.data.common]: Serialized dataset takes 0.34 MiB
[09/03 11:02:48 d2.evaluation.evaluator]: Start inference on 190 images
[09/03 11:02:52 d2.evaluation.evaluator]: Inference done 11/190. 0.3359 s / img. ETA=0:01:01
[09/03 11:02:57 d2.evaluation.evaluator]: Inference done 25/190. 0.3498 s / img. ETA=0:00:58
[09/03 11:03:02 d2.evaluation.evaluator]: Inference done 39/190. 0.3563 s / img. ETA=0:00:54
[09/03 11:03:07 d2.evaluation.evaluator]: Inference done 53/190. 0.3563 s / img. ETA=0:00:49
[09/03 11:03:12 d2.evaluation.evaluator]: Inference done 67/190. 0.3568 s / img. ETA=0:00:44
[09/03 11:03:18 d2.evaluation.evaluator]: Inference done 81/190. 0.3589 s / img. ETA=0:00:39
[09/03 11:03:23 d2.evaluation.evaluator]: Inference done 95/190. 0.3606 

OrderedDict([('bbox',
              {'AP': 30.028954867070958,
               'AP-aeroplane': 54.492201871100164,
               'AP-car': 16.518143172778075,
               'AP-chair': 21.81938670292362,
               'AP-cow': 50.554033712020306,
               'AP-person': 11.101298937862536,
               'AP-traffic_light': 25.68866480574103,
               'AP50': 45.9491811115916,
               'AP75': 31.514182270883616,
               'APl': 45.27411067071747,
               'APm': 41.31765659109794,
               'APs': 15.323416365796074}),
             ('segm',
              {'AP': 24.213020765128487,
               'AP-aeroplane': 39.783182438186024,
               'AP-car': 14.815570284965865,
               'AP-chair': 16.688287929509446,
               'AP-cow': 41.185953406719165,
               'AP-person': 9.33184244640012,
               'AP-traffic_light': 23.4732880849903,
               'AP50': 42.76290697621386,
               'AP75': 24.71819873424319,
    

In [12]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_train2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_train2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[09/03 10:53:04 d2.data.datasets.coco]: Loaded 689 images in COCO format from /content/drive/MyDrive/detectron2_500/train.json
[09/03 10:53:04 d2.data.common]: Serializing 689 elements to byte tensors and concatenating them all ...
[09/03 10:53:04 d2.data.common]: Serialized dataset takes 1.35 MiB
[09/03 10:53:04 d2.evaluation.evaluator]: Start inference on 689 images
[09/03 10:53:08 d2.evaluation.evaluator]: Inference done 11/689. 0.3771 s / img. ETA=0:04:18
[09/03 10:53:13 d2.evaluation.evaluator]: Inference done 26/689. 0.3547 s / img. ETA=0:03:58
[09/03 10:53:19 d2.evaluation.evaluator]: Inference done 41/689. 0.3529 s / img. ETA=0:03:52
[09/03 10:53:24 d2.evaluation.evaluator]: Inference done 55/689. 0.3551 s / img. ETA=0:03:48
[09/03 10:53:29 d2.evaluation.evaluator]: Inference done 69/689. 0.3566 s / img. ETA=0:03:45
[09/03 10:53:34 d2.evaluation.evaluator]: Inference done 83/689. 0.3586 s / img. ETA=0:03:41
[09/03 10:53:40 d2.evaluation.evaluator]: Inference done 97/689. 0.3588

OrderedDict([('bbox',
              {'AP': 89.12506004351113,
               'AP-aeroplane': 96.33243452550383,
               'AP-car': 87.36163965453434,
               'AP-chair': 89.67221230203245,
               'AP-cow': 89.07522564682145,
               'AP-person': 89.54169774255176,
               'AP-traffic_light': 82.76715038962288,
               'AP50': 98.76737248674019,
               'AP75': 97.03821634127807,
               'APl': 95.8040987066509,
               'APm': 92.76798578110366,
               'APs': 86.04362909014631}),
             ('segm',
              {'AP': 69.27450158343737,
               'AP-aeroplane': 70.86995521116954,
               'AP-car': 71.502296075192,
               'AP-chair': 67.47054019731053,
               'AP-cow': 65.32456225990904,
               'AP-person': 67.41861620597442,
               'AP-traffic_light': 73.06103955106875,
               'AP50': 97.53003416590172,
               'AP75': 83.48792895631885,
               '

In [13]:
import os
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2_500/output_detect/model_0065999.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_train2", cfg, False, output_dir="/content/drive/MyDrive/detectron2_500/output_detect/")
val_loader = build_detection_test_loader(cfg, "my_dataset_train2")
inference_on_dataset(trainer.model, val_loader, evaluator)

[09/03 10:57:37 d2.data.datasets.coco]: Loaded 689 images in COCO format from /content/drive/MyDrive/detectron2_500/train.json
[09/03 10:57:37 d2.data.common]: Serializing 689 elements to byte tensors and concatenating them all ...
[09/03 10:57:37 d2.data.common]: Serialized dataset takes 1.35 MiB
[09/03 10:57:37 d2.evaluation.evaluator]: Start inference on 689 images
[09/03 10:57:41 d2.evaluation.evaluator]: Inference done 11/689. 0.3713 s / img. ETA=0:04:14
[09/03 10:57:47 d2.evaluation.evaluator]: Inference done 26/689. 0.3531 s / img. ETA=0:03:57
[09/03 10:57:52 d2.evaluation.evaluator]: Inference done 41/689. 0.3518 s / img. ETA=0:03:51
[09/03 10:57:57 d2.evaluation.evaluator]: Inference done 55/689. 0.3542 s / img. ETA=0:03:47
[09/03 10:58:02 d2.evaluation.evaluator]: Inference done 69/689. 0.3555 s / img. ETA=0:03:44
[09/03 10:58:07 d2.evaluation.evaluator]: Inference done 83/689. 0.3576 s / img. ETA=0:03:40
[09/03 10:58:13 d2.evaluation.evaluator]: Inference done 97/689. 0.3579

OrderedDict([('bbox',
              {'AP': 89.12506004351113,
               'AP-aeroplane': 96.33243452550383,
               'AP-car': 87.36163965453434,
               'AP-chair': 89.67221230203245,
               'AP-cow': 89.07522564682145,
               'AP-person': 89.54169774255176,
               'AP-traffic_light': 82.76715038962288,
               'AP50': 98.76737248674019,
               'AP75': 97.03821634127807,
               'APl': 95.8040987066509,
               'APm': 92.76798578110366,
               'APs': 86.04362909014631}),
             ('segm',
              {'AP': 69.27450158343737,
               'AP-aeroplane': 70.86995521116954,
               'AP-car': 71.502296075192,
               'AP-chair': 67.47054019731053,
               'AP-cow': 65.32456225990904,
               'AP-person': 67.41861620597442,
               'AP-traffic_light': 73.06103955106875,
               'AP50': 97.53003416590172,
               'AP75': 83.48792895631885,
               '